# ADMMIN MODEL

In [ ]:
from sqlalchemy.dialects.mysql import base 
base.ischema_names['tinyint'] = base.BOOLEAN
base.ischema_names['mediumtext'] = base.TEXT 
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    String,
    Text,
    Time,
    DateTime,
    Date,
    Float,
    Boolean,
    ForeignKey,
    UniqueConstraint
)
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import inspect
from sqlalchemy import MetaData
from sqlalchemy import Table as _Table
from sqlalchemy.sql import select, and_
from sqlalchemy.sql import insert
from sqlalchemy.sql import update
from sqlalchemy.sql import delete
from sqlalchemy.sql import text
#from sqlalchemy.sql.expression import func

from sqlalchemy import func
import datetime
import json
import hashlib
import os
from sqlalchemy.types import TypeDecorator, Unicode
import sys
from sqlalchemy.engine.url import URL
import copy
#from sqlalchemy_utils import database_exists, create_database
import re
import pandas as pd
import numpy as np
import sqlalchemy as sa
import openpyxl as xl
from passlib.hash import pbkdf2_sha256
from passlib.context import CryptContext
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
from sqlalchemy_utils import database_exists, create_database

In [ ]:
link = 'tmp/EXEMPLE_20231112.parquet'

# DB

In [ ]:
database = 'ADMIN_DB'

In [ ]:
db_conf = {
    "drivername": "mysql+pymysql",
    "username": "root",
    "password": "1234",
    "host": "localhost",
    "port": 3306,
    "database": database
}
db_conf = {
    "drivername": "postgresql+psycopg2",
    "username": "postgres",
    "password": "1234",
    "host": "localhost",
    "port": 5432,
    "database": database
}
db_conf = {
    "drivername": "sqlite",
    "database": "database/{db}.db".format(db = database)
}
url = URL.create(**db_conf)
engine = create_engine(url, echo = False)
if not database_exists(engine.url):
    create_database(engine.url)

In [ ]:
url

# SESSION1

In [ ]:
Session = sessionmaker(bind = engine, autoflush = True)
session = Session()
Base = declarative_base()
metadata = MetaData()
metadata.reflect(engine)
inspector = inspect(engine)

cols = [str(f).replace('app' + '.','') for f in metadata.tables['app'].c]
cols

In [ ]:
_data = {}

# ADMIN DB MODELS

## Langs

In [ ]:
class Lang(Base):
    __tablename__ = 'lang'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Languages',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    lang_id     = Column(Integer, primary_key = True, autoincrement = True, comment = "Lang ID")
    lang        = Column(String(4), unique = True, nullable = False, comment = "Language")
    lang_desc   = Column(String(200), nullable = True, comment = "Description")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
now = datetime.datetime.now()
_data['lang'] = [
    {'lang_id': 1, 'lang': 'en', 'lang_desc': 'Languages', 'created_at': now, 'updated_at':  now, 'excluded':  False}
]

## Roles

In [ ]:
class Role(Base):
    __tablename__ = 'role'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Roles',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    role_id     = Column(Integer, primary_key = True, autoincrement = True, comment = "Role ID")
    role        = Column(String(20), nullable = False, unique = True, comment = "Role")
    role_desc   = Column(Text, nullable = True, comment = "Description")
    config      = Column(Text, nullable = True, comment = "Config")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
def sess_2_dict(sess):
    return "{'" + re.sub(r'\s+=', "': ", re.sub(r',\s+', ", '", sess)) + "}"

In [ ]:
_data['role'] = [
    {'role_id':  1, 'role':  'root', 'role_desc':  'Root role', 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## Users

In [ ]:
class User(Base):
    __tablename__ = 'user'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Users',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    user_id     = Column(Integer, primary_key = True, autoincrement = True, comment = "User ID")
    username    = Column(String(50), unique = True, nullable = False, comment = "Username")
    first_name  = Column(String(50), nullable = False, comment = "Fisrt Name")
    last_name   = Column(String(50), nullable = False, comment = "Last Name")
    email       = Column(String(50), unique = True, nullable = True, comment = "Email")
    phone       = Column(String(50), unique = True, nullable = True, comment = "Phone")
    password    = Column(String(200), nullable = False, comment = "Password")
    role_id     = Column(Integer, ForeignKey('role.role_id'), comment = "Default Role ID")
    lang_id     = Column(Integer, ForeignKey('lang.lang_id'), comment = "Lang ID")
    timezone    = Column(String(50), nullable = True, comment = "Timezone")
    attach_profile_pic = Column(String(200), nullable = True, comment = "Profile Picture")
    active      = Column(Boolean, default = True, comment = "Active")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
def verify_password(plain_password: str, hashed_password: str) -> bool:
        return pwd_context.verify(plain_password, hashed_password)
def get_password_hash(password: str) -> str:
    return pwd_context.hash(password)

In [ ]:
try:
    passw = get_password_hash('sdagfkjasgkfagsuy')
    print(passw)
except Exception as _err:# pylint: disable=broad-exception-caught
    print(str(_err))
    passw = ''

In [ ]:
#verify_password('sdagfkjasgkfagsuy', passw)

In [ ]:
passw = hashlib.md5('1234'.encode('utf-8')).hexdigest()
passw

In [ ]:
_data['user'] = [
    {'user_id':  1, 'username':  'root', 'first_name':  'Super', 'last_name':  'Admin', 'email':  'root@domain.com', 'password':  passw, 'role_id':  1, 'lang_id':  1, 'active':  True, 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## User Roles

In [ ]:
class UserRole(Base):
    __tablename__ = 'user_role'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'User Roles',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    user_role_id = Column(Integer, primary_key = True, autoincrement = True, comment = "User Role ID")
    user_id      = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    role_id      = Column(Integer, ForeignKey('role.role_id'), comment = "Role ID")
    active       = Column(Boolean, default = True, comment = "Active")
    created_at   = Column(DateTime, nullable = True, comment = "Created at")
    updated_at   = Column(DateTime, nullable = True, comment = "Updated at")
    excluded     = Column(Boolean, default = False, comment = "Excluded")

## Apps

In [ ]:
class App(Base):
    __tablename__ = 'app'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Applications',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    app_id      = Column(Integer, primary_key = True, autoincrement = True, comment = "App ID")
    app         = Column(String(20),  unique = True, nullable = False, comment = "App Name")
    app_desc    = Column(Text,        nullable = True,  comment = "Description")
    version     = Column(String(10),  nullable = False, comment = "Version")
    email       = Column(String(200), nullable = True,  comment = "Email")
    db          = Column(String(20),  nullable = False, comment = "Database")
    attach_logo = Column(String(200), nullable = True,  comment = "Logo") 
    config      = Column(Text,        nullable = True,  comment = "Config")
    user_id     = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at  = Column(DateTime,    nullable = True,  comment = "Created at")
    updated_at  = Column(DateTime,    nullable = True,  comment = "Updated at")
    excluded    = Column(Boolean,     default = False,  comment = "Excluded")
    # UniqueConstraint('app', 'version', name='uix_1')


In [ ]:
appConfig = {}
_data['app'] = [
    {'app_id':  1, 'app':  'ADMIN', 'app_desc':  'Admin', 'version':  '1.0.0', 'email':  None, 'db':  database, 'attach_logo':  None, 'config':  json.dumps(appConfig, ensure_ascii=False), 'user_id':  1, 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## Menus

In [ ]:
class Menu(Base):
    __tablename__ = 'menu'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Menus',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    menu_id     = Column(Integer, primary_key = True, autoincrement = True, comment = "Menu ID")
    menu        = Column(String(20), nullable = False, comment = "Menu")
    menu_desc   = Column(Text, nullable = True, comment = "Description")
    menu_icon   = Column(String(20), nullable = True, comment = "Icon")
    menu_order  = Column(Integer, nullable = True, comment = "Order")
    menu_config = Column(Text, nullable = True, comment = "Config")
    app_id      = Column(Integer, ForeignKey('app.app_id'), nullable = False, comment = "App ID")
    user_id     = Column(Integer, ForeignKey('user.user_id'), nullable = False, comment = "User ID")
    active      = Column(Boolean,  default = True, comment = "Active")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['menu'] = [
    {'menu_id':  1, 'menu':  'Admin', 'menu_desc':  'Admin', 'menu_icon':  'user-group', 'menu_order':  1, 'app_id':  1, 'user_id':  1, 'active':  True, 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'menu_id':  2, 'menu':  'Params', 'menu_desc':  None, 'menu_icon':  'adjustments', 'menu_order':  2, 'app_id':  1, 'user_id':  1, 'active':  True, 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'menu_id':  3, 'menu':  'ETL / Report / Base', 'menu_desc':  None, 'menu_icon':  'circle-stack', 'menu_order':  3, 'app_id':  1, 'user_id':  1, 'active':  True, 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## Tables

In [ ]:
class Table(Base):
    __tablename__ = 'table'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine'   : 'InnoDB',
        'comment'        : 'Tables',
        'mysql_charset'  : 'utf8',
        'sqlite_autoincrement': True
    }
    table_id     = Column(Integer, primary_key = True, autoincrement = True, comment = "Table ID")
    table        = Column(String(50), nullable = False, comment = "Table")
    table_desc   = Column(String(200), nullable = True, comment = "Description")
    db           = Column(String(50), nullable = True, comment = "Database")
    requires_rla = Column(Boolean,  default  = False, comment = "Requires Row Level Access")
    user_id      = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at   = Column(DateTime, nullable = True, comment = "Created at")
    updated_at   = Column(DateTime, nullable = True, comment = "Updated at")
    excluded     = Column(Boolean, default = False, comment = "Excluded")

## MenuTable

In [ ]:
class MenuTable(Base):
    __tablename__ = 'menu_table'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine'   : 'InnoDB',
        'comment'        : 'Menu Tables',
        'mysql_charset'  : 'utf8',
        'sqlite_autoincrement': True
    }
    menu_table_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Menu Table ID")
    menu_id       = Column(Integer, ForeignKey('menu.menu_id'),   comment = "Menu ID")
    table_id      = Column(Integer, ForeignKey('table.table_id'), comment = "Table ID")
    app_id        = Column(Integer, ForeignKey('app.app_id'),     comment = "App ID")
    user_id       = Column(Integer, ForeignKey('user.user_id'),   comment = "User ID")
    active        = Column(Boolean,  default  = True,             comment = "Active")
    requires_rla  = Column(Boolean,  default  = False,            comment = "Requires Row Level Access")
    menu_table_cnf= Column(Text,     nullable = True,             comment = "Config")
    created_at    = Column(DateTime, nullable = True,             comment = "Created at")
    updated_at    = Column(DateTime, nullable = True,             comment = "Updated at")
    excluded      = Column(Boolean,  default  = False,            comment = "Excluded")

## RoleApp

In [ ]:
class RoleApp(Base):
    __tablename__ = 'role_app'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine'   : 'InnoDB',
        'comment'        : 'Role Apps',
        'mysql_charset'  : 'utf8',
        'sqlite_autoincrement': True
    }
    role_app_id  = Column(Integer, primary_key = True, autoincrement = True, comment = "Role App ID")
    role_id      = Column(Integer, ForeignKey('role.role_id'), comment = "Role ID")
    app_id       = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    access       = Column(Boolean, default = True, comment = "Access")
    user_id      = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at   = Column(DateTime, nullable = True, comment = "Created at")
    updated_at   = Column(DateTime, nullable = True, comment = "Updated at")
    excluded     = Column(Boolean, default = False, comment = "Excluded")

## Role App Menus

In [ ]:
class RoleAppMenu(Base):
    __tablename__ = 'role_app_menu'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Role App Menus',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    role_app_menu_id  = Column(Integer, primary_key = True, autoincrement = True, comment = "Role App Menu ID")
    role_id      = Column(Integer, ForeignKey('role.role_id'), comment = "Role ID")
    app_id       = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    menu_id      = Column(Integer, ForeignKey('menu.menu_id'), comment = "Menu ID")
    access       = Column(Boolean, default = True, comment = "Access")
    user_id      = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at   = Column(DateTime, nullable = True, comment = "Created at")
    updated_at   = Column(DateTime, nullable = True, comment = "Updated at")
    excluded     = Column(Boolean, default = False, comment = "Excluded")

## Role App Menu Tables

In [ ]:
class RoleAppMenuTable(Base):
    __tablename__ = 'role_app_menu_table'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Role App Menu Tables',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    role_app_menu_table_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Role App Menu Table ID")
    role_id      = Column(Integer, ForeignKey('role.role_id'), comment = "Role ID")
    app_id       = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    menu_id      = Column(Integer, ForeignKey('menu.menu_id'), comment = "Menu ID")
    table_id     = Column(Integer, ForeignKey('table.table_id'), comment = "Table ID")
    create       = Column(Boolean, default = False, comment = "Create")
    read         = Column(Boolean, default = False, comment = "Read")
    update       = Column(Boolean, default = False, comment = "Update")
    delete       = Column(Boolean, default = False, comment = "Delete")
    user_id      = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at   = Column(DateTime, nullable = True, comment = "Created at")
    updated_at   = Column(DateTime, nullable = True, comment = "Updated at")
    excluded     = Column(Boolean, default = False, comment = "Excluded")

## Users Logs

In [ ]:
class UserLog(Base):
    __tablename__ = 'user_log'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'User Logs',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    user_log_id = Column(Integer, primary_key = True, autoincrement = True, comment = "User Log ID")
    user_id     = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    action      = Column(String(200), nullable = False, comment = "Action")
    req_ip      = Column(String(200), nullable = True, comment = "Request IP")
    req_at      = Column(DateTime, nullable = True, comment = "Request at")
    req_data    = Column(Text, nullable = True, comment = "Request Data")
    res_at      = Column(DateTime, nullable = True, comment = "Response at")
    res_type    = Column(String(200), nullable = True, comment = "Response Type")
    res_msg     = Column(String(500), nullable = True, comment = "Response Message")
    res_data    = Column(Text, nullable = True, comment = "Request Data")
    table       = Column(String(200), nullable = True, comment = "Table")
    db          = Column(String(200), nullable = True, comment = "Database")
    row_id      = Column(Integer, nullable = True, comment = "Database")
    app_id      = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    # old_data    = Column(Text, nullable = True, comment = "Old Data")
    new_data    = Column(Text, nullable = True, comment = "New Data")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

## Custom Table

In [ ]:
class CustomTable(Base):
    __tablename__ = 'custom_table'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Custom Table',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    custom_table_id = Column(Integer, primary_key = True, autoincrement = True, comment = "User Log ID")
    table       = Column(String(200), nullable = True, comment = "Table")
    db          = Column(String(200), nullable = True, comment = "Database")
    config      = Column(Text, nullable = True, comment = "Config")
    app_id      = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    user_id     = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

## Custom Form

In [ ]:
class CustomForm(Base):
    __tablename__ = 'custom_form'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Custom Form',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    custom_form_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Custom Form ID")
    table       = Column(String(200), nullable = True, comment = "Table")
    db          = Column(String(200), nullable = True, comment = "Database")
    config      = Column(Text, nullable = True, comment = "Config")
    app_id      = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    user_id     = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

## Role Row Level Access

In [ ]:
class RoleRowLevelAccess(Base):
    __tablename__ = 'role_row_level_access'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Role Row Level Access',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    role_row_level_access_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Role Row Level Access ID")
    role_id     = Column(Integer,  ForeignKey('role.role_id'), comment = "Role ID")
    row_id      = Column(Integer, nullable = False, comment = "Row ID")
    table_id    = Column(Integer, ForeignKey('table.table_id'), comment = "Table ID")
    table       = Column(String(200), nullable = False, comment = "Table")
    db          = Column(String(200), nullable = False, comment = "Database")
    user_id     = Column(Integer,  ForeignKey('user.user_id'), comment = "User ID")
    app_id      = Column(Integer,  ForeignKey('app.app_id'), comment = "App ID")
    read        = Column(Boolean,  default  = False, comment = "Read")
    update      = Column(Boolean,  default  = False, comment = "Update")
    delete      = Column(Boolean,  default  = False, comment = "Delete")
    share       = Column(Boolean,  default  = False, comment = "Share")
    created_at  = Column(DateTime, nullable = True,  comment = "Created at")
    updated_at  = Column(DateTime, nullable = True,  comment = "Updated at")
    excluded    = Column(Boolean,  default  = False, comment = "Excluded")

## Column Level Access

In [ ]:
class ColumnLevelAccess(Base):
    __tablename__ = 'column_level_access'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Column Level Access',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    column_level_access_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Column Level Access ID")
    column      = Column(Integer, nullable = False, comment = "Column")
    table_id    = Column(Integer, ForeignKey('table.table_id'), comment = "Table ID")
    table       = Column(String(200), nullable = False, comment = "Table")
    db          = Column(String(200), nullable = False, comment = "Database")
    user_id     = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id      = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    create       = Column(Boolean, default = False, comment = "Create")
    read        = Column(Boolean, default = False, comment = "Read")
    update      = Column(Boolean, default = False, comment = "Update")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

## Row Level Access

In [ ]:
class RowLevelAccess(Base):
    __tablename__ = 'row_level_access'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Row Level Access',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    row_level_access_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Row Level Access ID")
    row_id      = Column(Integer, nullable = False, comment = "Row ID")
    table_id    = Column(Integer, ForeignKey('table.table_id'), comment = "Table ID")
    table       = Column(String(200), nullable = False, comment = "Table")
    db          = Column(String(200), nullable = False, comment = "Database")
    user_id     = Column(Integer,  ForeignKey('user.user_id'), comment = "User ID")
    app_id      = Column(Integer,  ForeignKey('app.app_id'), comment = "App ID")
    read        = Column(Boolean,  default  = False, comment = "Read")
    update      = Column(Boolean,  default  = False, comment = "Update")
    delete      = Column(Boolean,  default  = False, comment = "Delete")
    share       = Column(Boolean,  default  = False, comment = "Share")
    created_at  = Column(DateTime, nullable = True,  comment = "Created at")
    updated_at  = Column(DateTime, nullable = True,  comment = "Updated at")
    excluded    = Column(Boolean,  default  = False, comment = "Excluded")

## Translate Table

In [ ]:
class TranslateTable(Base):
    __tablename__ = 'translate_table'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Translate Table',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    transl_tbl_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Translate Table ID")
    table_org_desc  = Column(String(200), nullable = False, comment = "Table Org. Desc")
    table_transl_desc  = Column(String(200), nullable = False, comment = "Table Transl. Desc")
    table       = Column(String(200), nullable = False, comment = "Table")
    db          = Column(String(200), nullable = False, comment = "Database")
    lang        = Column(String(5), nullable = False, comment = "Lang")
    user_id     = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id      = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at  = Column(DateTime, nullable = True, comment = "Created at")
    updated_at  = Column(DateTime, nullable = True, comment = "Updated at")
    excluded    = Column(Boolean, default = False, comment = "Excluded")

## Translate Table Fields

In [ ]:
class TranslateTableField(Base):
    __tablename__ = 'translate_table_field'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Translate Table Fields',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    transl_tbl_field_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Translate Table Field ID")
    field_org_desc      = Column(String(200), nullable = False, comment = "Field Org. Desc")
    field_transl_desc   = Column(String(200), nullable = False, comment = "Field Transl. Desc")
    field               = Column(String(200), nullable = False, comment = "Field")
    table               = Column(String(200), nullable = False, comment = "Table")
    db                  = Column(String(200), nullable = False, comment = "Database")
    lang                = Column(String(5),   nullable = False, comment = "Lang")
    user_id             = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id              = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at          = Column(DateTime,    nullable = True, comment = "Created at")
    updated_at          = Column(DateTime,    nullable = True, comment = "Updated at")
    excluded            = Column(Boolean,     default = False, comment = "Excluded")

# REPORTS / BASES

## Periodicity

In [ ]:
class Periodicity(Base):
    __tablename__ = 'periodicity'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Periodicity',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    periodicity_id  = Column(Integer, primary_key = True, autoincrement = True, comment = "Periodicity ID")
    periodicity     = Column(String(100), unique = True, nullable = False, comment = "Periodicity")
    created_at      = Column(DateTime, nullable = True, comment = "Created at")
    updated_at      = Column(DateTime, nullable = True, comment = "Updated at")
    excluded        = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['periodicity'] = [
    {'periodicity_id':  1, 'periodicity':  'Daily', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'periodicity_id':  2, 'periodicity':  'Monthly', 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## ETL / REPORT / BASE

In [ ]:
class ETLReportBase(Base):
    __tablename__ = 'etl_report_base'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'ETL / Report / Base',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_report_base_id    = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_report_base       = Column(String(200), unique = True, nullable = False, comment = "ETL / Report / Base")
    etl_report_base_desc  = Column(Text, nullable = True, comment = "Description")   
    attach_etl_rbase_doc  = Column(String(200), nullable = True, comment = "Documentation")
    periodicity_id        = Column(Integer, ForeignKey('periodicity.periodicity_id'), comment = "Periodicity")
    database              = Column(String(200), nullable = False, comment = "Database") 
    includes_data_quality = Column(Boolean, default = False, comment = "Includes Data Quality")
    includes_data_reconci = Column(Boolean, default = False, comment = "Includes Data Reconc.")
    includes_exports      = Column(Boolean, default = True, comment = "Includes Exports")
    includes_notify       = Column(Boolean, default = True, comment = "Includes Notefy")
    etl_report_base_conf  = Column(Text, nullable = True, comment = "Config")
    active                = Column(Boolean, default = True, comment = "Active")
    user_id               = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at            = Column(DateTime, nullable = True, comment = "Created at")
    updated_at            = Column(DateTime, nullable = True, comment = "Updated at")
    excluded              = Column(Boolean, default = False, comment = "Excluded")

## InputType

In [ ]:
class InputType(Base):
    __tablename__ = 'input_type'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Input Type',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    input_type_id  = Column(Integer, primary_key = True, autoincrement = True, comment = "Input Type ID")
    input_type     = Column(String(100), unique = True, nullable = False, comment = "Input Type")
    created_at     = Column(DateTime, nullable = True, comment = "Created at")
    updated_at     = Column(DateTime, nullable = True, comment = "Updated at")
    excluded       = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['input_type'] = [
    {'input_type_id':  1, 'input_type':  'Aux', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'input_type_id':  2, 'input_type':  'Main', 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## SourceType

In [ ]:
class SourceType(Base):
    __tablename__ = 'source_type'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Source Type',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    source_type_id = Column(Integer, primary_key = True, autoincrement = True, comment = "Source Type ID")
    source_type    = Column(String(100), unique = True, nullable = False, comment = "Souce Type")
    created_at     = Column(DateTime, nullable = True, comment = "Created at")
    updated_at     = Column(DateTime, nullable = True, comment = "Updated at")
    excluded       = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['source_type'] = [
    {'source_type_id':  1, 'source_type':  'File', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'source_type_id':  2, 'source_type':  'Database', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'source_type_id':  3, 'source_type':  'FTP', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'source_type_id':  4, 'source_type':  'eMail', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'source_type_id':  5, 'source_type':  'FileSystem', 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## INPUT

In [ ]:
class ETLReportBaseInput(Base):
    __tablename__ = 'etl_rbase_input'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Inputs',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rbase_input_id    = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rbase_input       = Column(String(200), nullable = False, comment = "Input")
    etl_rbase_input_desc  = Column(Text, nullable = True, comment = "Input Description")
    etl_report_base_id    = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), nullable = False, comment = "ETL / Report / Base ID")
    input_type_id         = Column(Integer, ForeignKey('input_type.input_type_id'), comment = "Input Type ID")
    save_only_temp        = Column(Boolean, default = True, comment = "Save only temp")
    replace_existing_data = Column(Boolean, default = False, comment = "Replace existing data")
    check_ref_date        = Column(Boolean, default = False, comment = "Check ref. date")
    ref_date_field        = Column(String(200), nullable = True, comment = "Ref. date field")
    date_format_org       = Column(String(200), nullable = True, comment = "Date Format in origin")
    other_date_fields     = Column(String(200), nullable = True, comment = "Other Date Fields")
    ref_id_keys           = Column(String(200), nullable = True, comment = "Reference / Id Keys")
    last_update_date_field= Column(String(200), nullable = True, comment = "Last update date field")
    incremental_extract   = Column(Boolean, default = False, comment = "Incremental Extraction")
    destination_table     = Column(String(200), nullable = False, comment = "Destination Table")
    database              = Column(String(200), nullable = False, comment = "Database")
    allow_import          = Column(Boolean, default = True, comment = "Allow Import")
    multiple_sheets       = Column(Boolean, default = False, comment = "Multiple Sheets")
    specific_sheets       = Column(String(200), nullable = True, comment = "Specific Sheets")
    specific_range        = Column(String(200), nullable = True, comment = "Specific Range")
    columns_to_import     = Column(String(200), nullable = True, comment = "Columns to Import")
    txt_fix_format_layout = Column(String(200), nullable = True, comment = "Text with fixed format Layout")
    headers               = Column(String(200), nullable = True, comment = "Headers")
    spreadsheet_forms     = Column(Boolean, default = False, comment = "Spreadsheet Forms")
    spreadsheet_forms_map = Column(String(200), nullable = True, comment = "Spreadsheet Forms Map")
    etl_rbase_input_conf  = Column(Text, nullable = True, comment = "Config")
    active                = Column(Boolean, default = True, comment = "Active")
    user_id               = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at            = Column(DateTime, nullable = True, comment = "Created at")
    updated_at            = Column(DateTime, nullable = True, comment = "Updated at")
    excluded              = Column(Boolean, default = False, comment = "Excluded")

## Output Type

In [ ]:
class OutputType(Base):
    __tablename__ = 'output_type'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Output Type',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    output_type_id  = Column(Integer, primary_key = True, autoincrement = True, comment = "Output Type ID")
    output_type     = Column(String(100), unique = True, nullable = False, comment = "Souce Type")
    created_at      = Column(DateTime, nullable = True, comment = "Created at")
    updated_at      = Column(DateTime, nullable = True, comment = "Updated at")
    excluded        = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['output_type'] = [
    {'output_type_id':  1, 'output_type':  'Table', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'output_type_id':  2, 'output_type':  'View', 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## OUTPUT

In [ ]:
class ETLReportBaseOutput(Base):
    __tablename__ = 'etl_rbase_output'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Outputs',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rbase_output_id    = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rbase_output       = Column(String(200), unique = False, nullable = False, comment = "Output")
    etl_rbase_output_desc  = Column(Text, nullable = True, comment = "Output Description")
    etl_report_base_id     = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), nullable = False, comment = "ETL / Report / Base ID")
    output_type_id         = Column(Integer, ForeignKey('output_type.output_type_id'), comment = "Input Type ID")
    date_field             = Column(String(200), nullable = True, comment = "Date field")
    date_field_format      = Column(String(200), nullable = True, comment = "Date Format")
    destination_table      = Column(String(200), nullable = False, comment = "Destination Table")
    database               = Column(String(200), nullable = False, comment = "Database")
    append_it              = Column(Boolean, default = False, comment = "Append")
    output_order           = Column(Integer, nullable = True, comment = "Field Order")
    etl_rbase_output_conf  = Column(Text, nullable = True, comment = "Config")
    active                 = Column(Boolean, default = True, comment = "Active")
    user_id                = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                 = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at             = Column(DateTime, nullable = True, comment = "Created at")
    updated_at             = Column(DateTime, nullable = True, comment = "Updated at")
    excluded               = Column(Boolean, default = False, comment = "Excluded")

## OUTPUT FIELDS

In [ ]:
class ETLReportBaseOutputFields(Base):
    __tablename__ = 'etl_rb_output_field'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Output Fields',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rb_output_field_id    = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rb_output_field       = Column(String(200), nullable = False, comment = "Field")
    etl_rb_output_field_desc  = Column(Text, nullable = True, comment = "Field Description")
    etl_rbase_output_id       = Column(Integer, ForeignKey('etl_rbase_output.etl_rbase_output_id'), nullable = False, comment = "Output ID")
    sql_select                = Column(Text, nullable = False, comment = "SELECT")
    sql_from                  = Column(Text, nullable = True,  comment = "FROM")
    sql_join                  = Column(Text, nullable = True,  comment = "JOIN")
    sql_where                 = Column(Text, nullable = True,  comment = "WHERE")
    sql_group_by              = Column(Text, nullable = True,  comment = "GROUP BY")
    sql_order_by              = Column(Text, nullable = True,  comment = "ORDER BY")
    sql_window                = Column(Text, nullable = True,  comment = "WINDOW")
    sql_having                = Column(Text, nullable = True,  comment = "HAVING")
    field_order               = Column(Integer, nullable = True, comment = "Field Order")
    fields_used               = Column(Text, nullable = True, comment = "Fields Used")
    etl_report_base_id        = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), comment = "ETL / Report / Base ID")
    active                    = Column(Boolean, default = True, comment = "Active")
    user_id                   = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                    = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at                = Column(DateTime, nullable = True, comment = "Created at")
    updated_at                = Column(DateTime, nullable = True, comment = "Updated at")
    excluded                  = Column(Boolean, default = False, comment = "Excluded")

## DATA QUALITY

In [ ]:
class ETLReportBaseDataQuality(Base):
    __tablename__ = 'etl_rbase_quality'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Data Quality',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rbase_quality_id   = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rbase_quality      = Column(String(200), unique = False, nullable = False, comment = "Data Quality Rule")
    etl_rbase_quality_desc = Column(Text, nullable = True, comment = "Rule Description")
    etl_report_base_id     = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), nullable = False, comment = "ETL / Report / Base ID")
    sql_quality_check      = Column(Text, nullable = False, comment = "SQL Quality Check")
    sql_quality_fix        = Column(Text, nullable = True, comment = "SQL Quality Fix")
    comments               = Column(Text, nullable = True, comment = "Comments / Justifications")
    fields                 = Column(Text, nullable = True, comment = "Fields")
    tables                 = Column(Text, nullable = True, comment = "Tables")
    database               = Column(String(200), nullable = False, comment = "Database")
    etl_rbase_quality_conf = Column(Text, nullable = True, comment = "Rule Description")
    active                 = Column(Boolean, default = True, comment = "Active")
    user_id                = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                 = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at             = Column(DateTime, nullable = True, comment = "Created at")
    updated_at             = Column(DateTime, nullable = True, comment = "Updated at")
    excluded               = Column(Boolean, default = False, comment = "Excluded")

## RECONCILIATION

In [ ]:
class ETLReportBaseDataReconciliation(Base):
    __tablename__ = 'etl_rb_reconcilia'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Data Reconciliation',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rb_reconcilia_id   = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rb_reconcilia      = Column(String(200), unique = False, nullable = False, comment = "Data Reconciliation Rule")
    etl_rb_reconcilia_desc = Column(Text, nullable = True, comment = "Rule Description")
    etl_rb_reconc_template = Column(Text, nullable = True, comment = "Template")
    database               = Column(String(200), nullable = False, comment = "Database")
    active                 = Column(Boolean, default = True, comment = "Active")
    etl_report_base_id     = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), nullable = False, comment = "ETL / Report / Base ID")
    user_id                = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                 = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at             = Column(DateTime, nullable = True, comment = "Created at")
    updated_at             = Column(DateTime, nullable = True, comment = "Updated at")
    excluded               = Column(Boolean, default = False, comment = "Excluded")

## RECONCILIATION DEATILS

In [ ]:
class ETLReportBaseDataReconciDetail(Base):
    __tablename__ = 'etl_rb_reconc_dtail'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Data Reconciliation',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rb_reconc_dtail_id   = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rb_reconc_dtail      = Column(String(50), nullable = False, comment = "Variable Name")
    etl_rb_reconc_dtail_desc = Column(String(200), nullable = False, comment = "Var. Description")
    sql_query_val_1          = Column(Text, nullable = True, comment = "SQL Query Valor 1")
    sql_query_val_2          = Column(Text, nullable = True, comment = "SQL Query Valor 2")
    is_eval_formula          = Column(Boolean, default = True, comment = "Is Eval Formula")
    sql_reconcilia_query     = Column(Text, nullable = True, comment = "SQL / Formula Reconc")
    comments                 = Column(Text, nullable = True, comment = "Comments / Justifications")
    etl_rb_reconcilia_id     = Column(Integer, ForeignKey('etl_rb_reconcilia.etl_rb_reconcilia_id'), nullable = False, comment = "Data Reconciliation ID")
    etl_report_base_id       = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), comment = "ETL / Report / Base ID")
    active                   = Column(Boolean, default = True, comment = "Active")
    user_id                  = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                   = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at               = Column(DateTime, nullable = True, comment = "Created at")
    updated_at               = Column(DateTime, nullable = True, comment = "Updated at")
    excluded                 = Column(Boolean, default = False, comment = "Excluded")

## Export Type

In [ ]:
class ExportType(Base):
    __tablename__ = 'export_type'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Export Type',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    export_type_id  = Column(Integer, primary_key = True, autoincrement = True, comment = "Export Type ID")
    export_type     = Column(String(100), unique = True, nullable = False, comment = "Export Type")
    created_at      = Column(DateTime, nullable = True, comment = "Created at")
    updated_at      = Column(DateTime, nullable = True, comment = "Updated at")
    excluded        = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['export_type'] = [
    {'export_type_id':  1, 'export_type':  'File', 'created_at':  now, 'updated_at':  now, 'excluded':  False},
    {'export_type_id':  2, 'export_type':  'Template', 'created_at':  now, 'updated_at':  now, 'excluded':  False}
]

## EXPORTS

In [ ]:
class ETLReportBaseExport(Base):
    __tablename__ = 'etl_rbase_export'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Exports',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rbase_export_id    = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rbase_export       = Column(String(200), unique = False, nullable = False, comment = "Export")
    etl_rbase_export_desc  = Column(Text, nullable = True, comment = "Export Description")
    etl_report_base_id     = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), nullable = False, comment = "ETL / Report / Base ID")
    export_type_id         = Column(Integer, ForeignKey('export_type.export_type_id'), nullable = False, comment = "Export ID")
    attach_file_template   = Column(String(200), nullable = False, comment = "File name / Template")
    txt_fix_format_layout  = Column(Text, nullable = True, comment = "Text with fixed format Layout")
    txt_fix_format_header  = Column(Text, nullable = True, comment = "Text with fixed format Headers")
    etl_rbase_export_conf  = Column(Text, nullable = True, comment = "Config")
    etl_rbase_output_id    = Column(Integer, ForeignKey('etl_rbase_output.etl_rbase_output_id'), nullable = False, comment = "Output ID")
    database               = Column(String(200), nullable = False, comment = "Database")
    active                 = Column(Boolean, default = True, comment = "Active")
    ignore                 = Column(Boolean, default = False, comment = "Ignore")
    user_id                = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                 = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at             = Column(DateTime, nullable = True, comment = "Created at")
    updated_at             = Column(DateTime, nullable = True, comment = "Updated at")
    excluded               = Column(Boolean, default = False, comment = "Excluded")

## EXPORTS DETAILS

In [ ]:
class ETLReportBaseExportDetail(Base):
    __tablename__ = 'etl_rb_exp_dtail'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Export Details',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    etl_rb_exp_dtail_id    = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    etl_rb_exp_dtail       = Column(String(200), unique = False, nullable = False, comment = "Export Detail")
    etl_rb_exp_dtail_desc  = Column(Text, nullable = True, comment = "Export Detail Description")
    etl_rbase_export_id    = Column(Integer, ForeignKey('etl_rbase_export.etl_rbase_export_id'), nullable = False, comment = "Export ID")
    etl_report_base_id     = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), nullable = False, comment = "ETL / Report / Base ID")
    sql_export_query       = Column(Text, nullable = True, comment = "Export SQL Query")
    database               = Column(String(200), nullable = False, comment = "Database")
    # txt_fix_format_layout  = Column(Text, nullable = True, comment = "Text with fixed format Layout")
    dest_sheet_name        = Column(String(200), nullable = True, comment = "Dest. Sheet Name")
    dest_table_name        = Column(String(200), nullable = True, comment = "Dest. Table Name")
    etl_rb_exp_dtail_conf  = Column(Text, nullable = True, comment = "Config")
    active                 = Column(Boolean, default = True, comment = "Active")
    user_id                = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id                 = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at             = Column(DateTime, nullable = True, comment = "Created at")
    updated_at             = Column(DateTime, nullable = True, comment = "Updated at")
    excluded               = Column(Boolean, default = False, comment = "Excluded")

## LOGS

In [ ]:
class ETLReportBaseLogs(Base):
    __tablename__ = 'etl_report_base_log'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Logs ETL / Report / Base',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    log_id   = Column(Integer, primary_key=True, autoincrement=True, comment="ID Log")
    type     = Column(String(50), nullable=False, comment="Type")
    name     = Column(String(100), nullable=False, comment="Name")
    ref      = Column(Date, nullable=True, comment="Ref. Date")
    start    = Column(DateTime, nullable=True, comment="Started At")
    end      = Column(DateTime, nullable=True, comment="End At")
    timer    = Column(String(10), nullable=True, comment="Duration")
    success  = Column(Boolean, nullable=True, comment="Success")
    msg      = Column(Text, nullable=True, comment="Response")
    num_rows = Column(Integer, nullable=True, comment="Affected Rows")
    errors   = Column(Integer, nullable=True, comment="Errors")
    fixes    = Column(Integer, nullable=True, comment="Automated Fixes")
    fname    = Column(String(200), nullable=True, comment="Generated Files")
    html     = Column(Text, nullable=True, comment="Html")
    etl_report_base_id = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), comment = "ETL / Report / Base ID")
    user_id    = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at = Column(DateTime, nullable = True, comment = "Created at")
    updated_at = Column(DateTime, nullable = True, comment = "Updated at")
    excluded   = Column(Boolean, default = False, comment = "Excluded")

## NOTIFICATIONS

In [ ]:
class ETLReportBaseNotify(Base):
    __tablename__ = 'etl_rbase_notify'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Notify',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    notify_id      = Column(Integer, primary_key=True, autoincrement=True, comment="ID")
    notify_subject = Column(String(200), nullable=False, comment="Subject")
    notify_body    = Column(Text, nullable=False, comment="Body")
    notify_to      = Column(String(200), nullable=False, comment="Mail TO")
    notify_cc      = Column(String(200), nullable=True, comment="Mail CC")
    notify_attach_exports = Column(Boolean, nullable=True, default = False, comment="Attach Exports")
    notify_copy_exports_to = Column(Boolean, nullable=True, comment="Copy Exports")
    notify_copy_exports_path = Column(String(200), nullable=True, comment="Copy Exports Path")
    notify_conf    = Column(Text, nullable=True, comment="Conf")
    send_email     = Column(Boolean, default = True, comment = "Send Email")
    active         = Column(Boolean, default = True, comment = "Active")
    etl_report_base_id = Column(Integer, ForeignKey('etl_report_base.etl_report_base_id'), comment = "ETL / Report / Base ID")
    user_id        = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    created_at     = Column(DateTime, nullable = True, comment = "Created at")
    updated_at     = Column(DateTime, nullable = True, comment = "Updated at")
    excluded       = Column(Boolean, default = False, comment = "Excluded")

## QUERY MANEGMENT

In [ ]:
class ManageQuery(Base):
    __tablename__ = 'manage_query'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine' : 'InnoDB',
        'comment'      : 'Queries',
        'mysql_charset': 'utf8',
        'sqlite_autoincrement': True
    }
    manage_query_id    = Column(Integer, primary_key = True, autoincrement = True, comment = "ID")
    manage_query       = Column(String(200), unique = False, nullable = False, comment = "Query Desc")
    database           = Column(String(200), nullable = False, comment = "Database") 
    manage_query_conf  = Column(Text, nullable = True, comment = "Query Config")
    active             = Column(Boolean, default = True, comment = "Active")
    user_id            = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id             = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at         = Column(DateTime, nullable = True, comment = "Created at")
    updated_at         = Column(DateTime, nullable = True, comment = "Updated at")
    excluded           = Column(Boolean, default = False, comment = "Excluded")

# DASHBOARD

In [ ]:
class Dashboard(Base):
    __tablename__ = 'dashboard'
    __table_args__ = {
        'extend_existing': True,
        'mysql_engine'   : 'InnoDB',
        'comment'        : 'Dashboards',
        'mysql_charset'  : 'utf8',
        'sqlite_autoincrement': True
    }
    dashboard_id   = Column(Integer, primary_key = True, autoincrement = True, comment = "Dashboard ID")
    dashboard      = Column(String(200), comment = "Dashboard")
    dashboard_desc = Column(Text, nullable = True, comment = "Description")
    dashboard_conf = Column(Text, nullable = False, comment = "Conf / Params")
    order          = Column(Integer, nullable = True, comment = "Order")
    active         = Column(Boolean, default = True, comment = "Active")
    user_id        = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id         = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at     = Column(DateTime, nullable = True, comment = "Created at")
    updated_at     = Column(DateTime, nullable = True, comment = "Updated at")
    excluded       = Column(Boolean, default = False, comment = "Excluded")

# TASKS

#### Calendar

In [ ]:
class Calendar(Base):
    __tablename__ = 'calendar'
    __table_args__ = {
        'extend_existing'     : True,
        'mysql_engine'        : 'InnoDB',
        'comment'             : 'Calendar',
        'mysql_charset'       : 'utf8',
        'sqlite_autoincrement': True
    }
    calendar_id    = Column(Integer, primary_key=True, autoincrement=True, comment="Calendar ID")
    calendar       = Column(String(100), nullable=False, comment="Calendar")
    calendar_desc  = Column(Text, nullable=True, comment="Description")
    calendar_email = Column(String(200), nullable=True, comment="Email")
    calendar_color = Column(String(50), nullable=True, comment="Calendar Color")
    user_id        = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id         = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at     = Column(DateTime, nullable = True, comment = "Created at")
    updated_at     = Column(DateTime, nullable = True, comment = "Updated at")
    excluded       = Column(Boolean, default = False, comment = "Excluded")

#### TaskStatus

In [ ]:
class TaskStatus(Base):
    __tablename__ = 'task_status'
    __table_args__ = {
        'extend_existing'     : True,
        'mysql_engine'        : 'InnoDB',
        'comment'             : 'Status',
        'mysql_charset'       : 'utf8',
        'sqlite_autoincrement': True
    }
    task_status_id = Column(Integer, primary_key=True, autoincrement=True, comment="Status ID")
    task_status    = Column(String(10), nullable=False, comment="Status")
    excluded       = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['task_status'] = [
    {'task_status_id': 1, 'task_status':  'Atribuido', 'excluded': False},
    {'task_status_id': 2, 'task_status':  'Staterd', 'excluded': False},
    {'task_status_id': 3, 'task_status':  'On Standby', 'excluded': False},
    {'task_status_id': 4, 'task_status':  'Concluded', 'excluded': False}
]

#### RepeatType

In [ ]:
class RepeatType(Base):
    __tablename__ = 'repeat_type'
    __table_args__ = {
        'extend_existing'     : True,
        'mysql_engine'        : 'InnoDB',
        'comment'             : 'Type of repetition',
        'mysql_charset'       : 'utf8',
        'sqlite_autoincrement': True
    }
    repeat_type_id = Column(Integer, primary_key=True, autoincrement=True, comment="Type of repetition ID")
    repeat_type    = Column(String(10), nullable=False, comment="Type of Repetition")
    excluded       = Column(Boolean, default = False, comment = "Excluded")

In [ ]:
_data['repeat_type'] = [
    {'repeat_type_id': 1, 'repeat_type':  'Daily', 'excluded': False},
    {'repeat_type_id': 2, 'repeat_type':  'Weekly', 'excluded': False},
    {'repeat_type_id': 3, 'repeat_type':  'Monthly', 'excluded': False},
    {'repeat_type_id': 4, 'repeat_type':  'Anualy', 'excluded': False}
]

#### Task

In [ ]:
class Task(Base):
    __tablename__ = 'task'
    __table_args__ = {
        'extend_existing'     : True,
        'mysql_engine'        : 'InnoDB',
        'comment'             : 'Tasks',
        'mysql_charset'       : 'utf8',
        'sqlite_autoincrement': True
    }
    task_id           = Column(Integer, primary_key=True, autoincrement=True, comment="Task Id")
    task              = Column(String(100), nullable=False, comment="task")
    task_desc         = Column(Text, nullable=True, comment="Description")
    starts_at         = Column(DateTime, nullable=False, comment="Starts at")
    ends_at           = Column(DateTime, nullable=True, comment="Ends at")
    calendar_id       = Column(Integer, ForeignKey('calendar.calendar_id'), comment="Calendar ID")
    calendar_color    = Column(String(50), nullable=True, comment="Calendar Color")
    calendar_email    = Column(String(200), nullable=True, comment="Email")
    task_status_id    = Column(Integer, ForeignKey('task_status.task_status_id'), nullable=True, default=None, comment="Status ID")
    task_status       = Column(String(50), nullable=True, default=None, comment="Status")
    attach_task       = Column(String(200), nullable=True, comment="Attachment")
    # REPEATE
    repeat            = Column(Boolean, nullable=True, default=True, comment="Repeat")
    repeat_type_id    = Column(Integer, ForeignKey('repeat_type.repeat_type_id'), nullable=True, default=None, comment="Repeat Type ID")
    days_of_week      = Column(String(200), nullable=True, comment="Days of week") #daysOfWeek
    repeat_start_date = Column(Date, nullable=True, comment="Repeat start date") #startRecur
    repeat_start_time = Column(Time, nullable=True, comment="Repeat start time") #startTime
    repeat_end_date   = Column(Date, nullable=True, comment="Repeat end date")  #endRecur
    repeat_end_time   = Column(Time, nullable=True, comment="Repeat end time")  #endTime
    attendees         = Column(Text, nullable=True, comment="Attendees")
    active            = Column(Boolean, default = True, comment = "Active")
    user_id           = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id            = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at        = Column(DateTime, nullable = True, comment = "Created at")
    updated_at        = Column(DateTime, nullable = True, comment = "Updated at")
    excluded          = Column(Boolean, default = False, comment = "Excluded")

#### TaskTrack

In [ ]:
class TaskTrack(Base):
    __tablename__ = 'task_track'
    __table_args__ = {
        'extend_existing'     : True,
        'mysql_engine'        : 'InnoDB',
        'comment'             : 'Status Updates',
        'mysql_charset'       : 'utf8',
        'sqlite_autoincrement': True
    }
    task_track_id     = Column(Integer, primary_key=True, autoincrement=True, comment="Status Update ID")
    task_track        = Column(String(100), nullable=False, comment="Status Update")
    task_track_desc   = Column(Text, nullable=True, comment="Description")
    task_track_date   = Column(DateTime, nullable=False, comment="Date")
    task_status_id    = Column(Integer, ForeignKey('task_status.task_status_id'), nullable=True, default=None, comment="Status ID")
    attach_task_track = Column(String(200), nullable=True, comment="Attachment")
    task_id           = Column(Integer, ForeignKey('task.task_id'), comment="Task ID")
    user_id           = Column(Integer, ForeignKey('user.user_id'), comment = "User ID")
    app_id            = Column(Integer, ForeignKey('app.app_id'), comment = "App ID")
    created_at        = Column(DateTime, nullable = True, comment = "Created at")
    updated_at        = Column(DateTime, nullable = True, comment = "Updated at")
    excluded          = Column(Boolean, default = False, comment = "Excluded")

# DROP FIRST

In [ ]:
#engine.dispose()

In [ ]:
#session.close()
#ETLReportBase.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseInput.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseOutput.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseOutputFields.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseDataQuality.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseDataReconciDetail.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseDataReconciliation.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseNotify.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseExport.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseExportDetail.__table__.drop(bind = engine, checkfirst = True)
#ETLReportBaseLogs.__table__.drop(bind = engine, checkfirst = True)
#Task.__table__.drop(bind = engine, checkfirst = True)
ETLReportBaseOutputFields.__table__.drop(bind = engine, checkfirst = True)

# DROP ALL FIRST

In [ ]:
#session.close()
try:
    pass
    #Base.metadata.drop_all(bind = engine, checkfirst = True)
except Exception as e:
    print(str(e))

TAXY YYYYMMDD
joao.a.d.tavares91@gmail.com
~/Downloads/TaxiTest/YYYY/MM/DD

# CREATE ALL

In [ ]:
Base.metadata.create_all(bind = engine, checkfirst = True)

# ADD DATA

In [ ]:
#session.close()
tables = []
for cls in Base.__subclasses__():
    tables.append(cls.__tablename__)
#print(tables)

In [ ]:
_updated_tables = None # ['periodicity', 'etl_report_base', 'input_type', 'source_type', 'etl_rbase_input', 'output_type', 'etl_rbase_output', 'etl_rb_output_field', 'etl_rbase_quality', 'etl_rb_reconcilia', 'export_type', 'etl_rbase_export', 'etl_rb_exp_dtail', 'etl_report_base_log', 'etl_rbase_notify']
_updated_tables = None #['calendar', 'repeat_type', 'task_status', 'task', 'task_track']

In [ ]:
for table in _data:
    if not _updated_tables:
        pass
    elif len(_updated_tables) == 0:
        pass
    elif table not in _updated_tables:
        continue    
    print(table, '...')
    tbl = _Table(table, metadata, autoload_with = engine)
    with engine.connect() as conn_admin:
        try: 
            sql = insert(tbl).values(_data.get(table))
            result = conn_admin.execute(sql)
            print(table, result.rowcount, 'rows inserted')
            result.close()
            conn_admin.commit()
            conn_admin.close()
        except Exception as e:
            print(str(e)[0:100])

In [ ]:
tables = Base.metadata.tables
print(tables)

# REGISTER ALL THE TABLE AND THE COMMENTS / LABELS

In [ ]:
i = 1
j = 1
now = datetime.datetime.now()
_data = {}
for cls in Base.__subclasses__():
    if not _updated_tables:
        pass
    elif len(_updated_tables) == 0:
        pass
    elif cls.__tablename__ not in _updated_tables:
        continue
    args = cls.__table_args__
    if not _data.get('table'):
        _data['table'] = []
    _data['table'].append({
        'table': cls.__tablename__,
        'table_desc': args['comment'],
        'db': database,
        'user_id': 1,
        'created_at': now,
        'updated_at': now,
        'excluded': False
    })
    if not _data.get('translate_table'):
        _data['translate_table'] = []
    _data['translate_table'].append({
        'table_org_desc': args['comment'],
        'table_transl_desc': args['comment'],
        'table': cls.__tablename__,
        'db': database,
        'lang': 'en',
        'user_id': 1,
        'created_at': now,
        'updated_at': now,
        'excluded': False
    })
    for c in cls.__table__.columns:
        if not _data.get('translate_table_field'):
            _data['translate_table_field'] = []
        _data['translate_table_field'].append({
                'field_org_desc': c.comment,
                'field_transl_desc': c.comment,
                'field': str(c.name),
                'table': cls.__tablename__,
                'db': database,
                'lang': 'en',
                'user_id': 1,
                'created_at': now,
                'updated_at': now,
                'excluded': False
            })
        j += 1
    i += 1

In [ ]:
#print(_data)

In [ ]:
chk_first = True
for table in _data:
    print(table, '...')
    tbl = _Table(table, metadata, autoload_with = engine)
    sql = delete(tbl).where(tbl.c.db == database)
    with engine.connect() as conn_admin:
        if chk_first == True:
            for d in _data.get(table):
                sql = select(tbl.c)\
                    .select_from(tbl)\
                    .where(and_(
                        tbl.c.db == database,
                        tbl.c.table == d.get('table')
                    ))
                if table == 'translate_table_field':
                    sql = select(tbl.c)\
                        .select_from(tbl)\
                        .where(and_(
                            tbl.c.db == database,
                            tbl.c.table == d.get('table'),
                            tbl.c.field == d.get('field')
                        ))
                _curr = conn_admin.execute(sql).fetchone()
                if _curr:
                    # print('_curr:', _curr)
                    sql = update(tbl)\
                        .where(and_(
                            tbl.c.db == database,
                            tbl.c.table == d.get('table')
                        ))\
                        .values(d)
                    if table == 'translate_table_field':
                        sql = update(tbl)\
                            .where(and_(
                                tbl.c.db == database,
                                tbl.c.table == d.get('table'),
                                tbl.c.field == d.get('field')
                            )).values(d)
                    result = conn_admin.execute(sql)
                    print(d.get('table'), d.get('field'), result.rowcount, 'rows updated')
                    result.close()
                else:
                    sql = insert(tbl).values(d)
                    result = conn_admin.execute(sql)
                    print(table, result.rowcount, 'rows inserted')
                    result.close()
        else:
            result = conn_admin.execute(sql)
            print(table, result.rowcount, 'rows deleted')
            result.close()
            sql = insert(tbl).values(_data.get(table))
            result = conn_admin.execute(sql)
            print(table, result.rowcount, 'rows inserted')
            result.close()
        conn_admin.commit()
        conn_admin.close()

# COMMIT

In [ ]:
commit = True
#session.rollback()

In [ ]:
try:
    if commit == True:
        session.commit()
except Exception as e:
    print(str(e))

# CONFIG INTERFACE

In [ ]:
dash_conf = {
    "label": "dashboard",
    "tooltip": "dashboard_desc",
    "load_items": {
        "table": "dashboard",
        "tables": ["dashboard"]
    }
}
etl_conf = {
    "label": "etl_report_base",
    "tooltip": "etl_report_base_desc",
    "load_items": {
        "table": "etl_report_base",
        "tables": ["etl_report_base"]
    }
}
task_conf = {
    "label": "etl_report_base",
    "tooltip": "etl_report_base_desc",
    "load_items": {
        "table": "etl_report_base",
        "tables": ["etl_report_base"]
    }
}

In [ ]:
interface_conf = {
    'Dashboards': {
        'menu_icon': 'document-report',
        'menu_order': 0,
        'active': True,
        'menu_config': json.dumps(dash_conf),
        'tables': [
            'dashboard'
        ]
    },
    'Admin': {
        'menu_icon': 'web',
        'menu_order': 1,
        'active': True,
        'tables': [
            'app',
            'menu',
            'role',
            'user',
            'user_log',
            'custom_table',
            'custom_form',
            'table',
        ]
    },
    'Params':{
        'menu_icon': 'settings',
        'menu_order': 2,
        'active': True,
        'tables': [
            'lang'
        ]
    },
    'ETL / Report / Base':{
        'menu_icon': 'circle-stack',
        'menu_order': 3,
        'active': True,
        #'menu_config': json.dumps(etl_conf),
        'tables': [
            {'table': 'periodicity', 'active': False}, 
            'etl_report_base',
            'manage_query',
            {'table': 'input_type', 'active': False},
            {'table': 'source_type', 'active': False},
            {'table': 'etl_rbase_input', 'active': False},
            {'table': 'output_type', 'active': False},
            {'table': 'etl_rbase_output', 'active': False},
            {'table': 'etl_rb_output_field', 'active': False}, 
            {'table': 'etl_rbase_quality', 'active': False},
            {'table': 'etl_rb_reconcilia', 'active': False},
            {'table': 'etl_rb_reconc_dtail', 'active': False}, 
            {'table': 'export_type', 'active': False},
            {'table': 'etl_rbase_export', 'active': False},
            {'table': 'etl_rb_exp_dtail', 'active': False}, 
            {'table': 'etl_report_base_log', 'active': False},
            {'table': 'etl_rbase_notify', 'active': False},
        ]
    },
    'Tasks':{
        'menu_icon': 'calendar-days',
        '_menu_icon': 'rectangle-stack',
        'menu_order': 4,
        'active': True,
        # 'menu_config': json.dumps(task_conf),
        'tables': [
            {'table': 'calendar', 'active': False},
            {'table': 'task_status', 'active': False},
            {'table': 'repeat_type', 'active': False},
            {'table': 'task', 'active': True}, 
            {'table': 'task_track', 'active': False}
        ]
    } 
}

In [ ]:
def _run(interface_conf):
    tbl_app = _Table('app', metadata, autoload_with = engine)
    tbl_menu = _Table('menu', metadata, autoload_with = engine)
    tbl_table = _Table('table', metadata, autoload_with = engine)
    tbl_menu_table = _Table('menu_table', metadata, autoload_with = engine)
    sql = select(tbl_app.c).select_from(tbl_app).where(tbl_app.c.db == database)
    now = datetime.datetime.now()
    with engine.connect() as conn_admin:
        app = conn_admin.execute(sql).fetchone()
        for m in interface_conf:
            if interface_conf[m].get('active') == True:
                sql = select(tbl_menu.c)\
                        .select_from(tbl_menu)\
                        .where(and_(
                            tbl_menu.c.menu == m,
                            tbl_menu.c.app_id == app.app_id
                        ))
                menu = conn_admin.execute(sql).fetchone()
                # print(menu)
                if menu:
                    print(m, menu.menu_id)
                else:
                    data = {
                        'menu':  m,
                        'menu_desc': m,
                        'menu_icon': interface_conf[m].get('menu_icon'),
                        'active': interface_conf[m].get('active'),
                        'menu_order': interface_conf[m].get('menu_order'),
                        'menu_config': interface_conf[m].get('menu_config'),
                        'app_id': app.app_id,
                        'user_id': 1,
                        'created_at': now,
                        'updated_at': now,
                        'excluded': False
                    }
                    sql2 = insert(tbl_menu).values(data)
                    result = conn_admin.execute(sql2)
                    result.close()
                    menu = conn_admin.execute(sql).fetchone()  
                    print(m, 'Adicionado: ', menu.menu_id)
                # MENU TABELA
                if interface_conf[m].get('tables'):
                    for t in interface_conf[m].get('tables'):
                        _active = True
                        if type(t) == dict:
                            _tbl = t.get('table')
                            _active = t.get('active')
                        else:
                            _tbl = t
                        sql = select(tbl_table.c)\
                                .select_from(tbl_table)\
                                .where(and_(
                                    tbl_table.c.table == _tbl,
                                    tbl_table.c.db == database
                                ))
                        table = conn_admin.execute(sql).fetchone()
                        # print(_tbl, database, table)
                        try:
                            # print(_tbl, database, table, table)
                            sql = select(tbl_menu_table.c)\
                                    .select_from(tbl_menu_table)\
                                    .where(and_(
                                        tbl_menu_table.c.table_id == table.table_id,
                                        tbl_menu_table.c.menu_id == menu.menu_id,
                                        tbl_menu_table.c.app_id == app.app_id
                                    ))
                            menu_table = conn_admin.execute(sql).fetchone()
                            if not menu_table:
                                data = {
                                    'menu_id': menu.menu_id,
                                    'table_id': table.table_id,
                                    'app_id': app.app_id,
                                    'active': _active,
                                    'user_id': 1,
                                    'created_at': now,
                                    'updated_at': now,
                                    'excluded': False
                                }
                                sql2 = insert(tbl_menu_table).values(data)
                                result = conn_admin.execute(sql2)
                                result.close()
                        except Exception as e:
                            print(_tbl, table, str(e))
        conn_admin.commit()

In [ ]:
#_run(interface_conf)